In [2]:
import pandas as pd

employee_survey = pd.read_csv('data/raw/employee_survey_data.csv')
employee_survey.head(1)

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0


In [3]:
manager_survey = pd.read_csv('data/raw/manager_survey_data.csv')
manager_survey.head(1)

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3


In [4]:
general_data = pd.read_csv('data/raw/general_data.csv')
general_data.head(1)

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0


In [5]:
import zipfile
import os

def get_in_out_time(DATA_DIR: str = 'data/raw') -> tuple[pd.DataFrame, pd.DataFrame]:
    join = os.path.join
    with zipfile.ZipFile(join(DATA_DIR,'in_out_time.zip'), 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)
    in_time = pd.read_csv(join(DATA_DIR,'in_time.csv'))
    out_time = pd.read_csv(join(DATA_DIR, 'out_time.csv'))

    # rename first column to 'EmployeeID'
    in_time.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)
    out_time.rename(columns={'Unnamed: 0': 'EmployeeID'}, inplace=True)

    return in_time, out_time

in_time, out_time = get_in_out_time()
in_time.shape, out_time.shape

((4410, 262), (4410, 262))

In [23]:
def merge_times(in_t: pd.DataFrame, out_t: pd.DataFrame):
    # out_t and in_t have the same layout:
    #  - EmployeeID
    #  - 2015-01-01_x
    #  - 2015-01-02_x
    #  - ...

    # We want to merge them on EmployeeID and Date labels (labels are the same)
    columns = in_t.columns.to_list()

    in_out_times = pd.DataFrame(columns=columns)
    in_out_times['EmployeeID'] = in_t['EmployeeID']

    for c in columns[1:]:
        in_out_times[c] = pd.to_datetime(out_t[c]) - pd.to_datetime(in_t[c])
    return in_out_times


in_out_time = merge_times(in_time, out_time)

PROCESS_DIR = 'data/processed'
if not os.path.exists(PROCESS_DIR):
    os.makedirs(PROCESS_DIR)

in_out_time.to_csv(os.path.join(PROCESS_DIR, 'worked_time.csv'), index=False)
in_out_time.head(1)


,EmployeeID,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,...,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaT,0 days 07:12:30,0 days 07:11:23,0 days 07:24:39,0 days 07:00:24,0 days 07:17:23,0 days 07:29:04,0 days 07:15:46,0 days 07:49:52,...,NaT,0 days 07:20:21,0 days 07:23:45,0 days 06:30:17,0 days 07:35:47,NaT,0 days 07:46:26,0 days 07:18:54,0 days 07:46:44,0 days 07:04:49
